In [1]:
%load_ext sql
%sql sqlite:///chinook.db

## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (судить по количеству купленных треков).

In [2]:
%%sql
SELECT Country
FROM albums

INNER JOIN tracks, invoice_items, invoices, customers
ON albums.AlbumID = tracks.AlbumID and tracks.TrackID = invoice_items.TrackID and
invoice_items.InvoiceID = invoices.InvoiceID and invoices.CustomerID = customers.CustomerID
WHERE albums.Title = "Iron Maiden"
GROUP BY Country
ORDER BY COUNT(Country) DESC
LIMIT 1



 * sqlite:///chinook.db
Done.


Country
USA


## Задание 2
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(цена альбома - сумма цен треков)*.

In [3]:
%%sql
SELECT AlbumID, Title,
       AlbumUnitPrice, AlbumsTime
FROM (
  SELECT SUM(UnitPrice) as AlbumUnitPrice,
         SUM(Milliseconds) as AlbumsTime,
         Title,
         albums.AlbumID
  FROM albums
  INNER JOIN tracks
  ON albums.AlbumID = tracks.AlbumID
  GROUP BY albums.AlbumID
)
WHERE AlbumUnitPrice > (SELECT AVG(AlbumUnitPrice) FROM (
  SELECT SUM(UnitPrice) as AlbumUnitPrice,
         albums.AlbumID
  FROM albums
  INNER JOIN tracks
  ON albums.AlbumID = tracks.AlbumID
  GROUP BY albums.AlbumID
  ) )
ORDER BY AlbumsTime
LIMIT 1

 * sqlite:///chinook.db
Done.


AlbumID,Title,AlbumUnitPrice,AlbumsTime
12,BackBeat Soundtrack,11.88,1615722


## Задание 3
Вывести общую стоимость треков, приобретённых компаниями (только клиентами из конкретных компаний), которые обслуживались сотрудниками, нанятыми после 3 августа 2002 года.

In [4]:
%%sql
SELECT SUM(UnitPrice) as "Общая сумма треков"
FROM (
  SELECT * FROM invoice_items
  INNER JOIN invoices, customers, employees, tracks
  ON invoice_items.InvoiceID = invoices.InvoiceID and
    invoices.CustomerID = customers.CustomerID and
    customers.SupportRepID = employees.EmployeeID and
    invoice_items.TrackID = tracks.TrackID
  WHERE Company is not NULL and HireDate > "2002-08-03")

 * sqlite:///chinook.db
Done.


Общая сумма треков
230.72000000000062


## Задание 4
Для каждого альбома определить все жанры песен, которые в нём есть и количество песен по каждому жанру (формат вывода: альбом, жанр, количество).
Выводить только те альбомы, где жанров больше одного.

In [5]:
%%sql
SELECT AlbumsName, GenresName, COUNT(AlbumsName)

FROM (
  SELECT albums.Title as AlbumsName, genres.Name as GenresName
  FROM tracks
  INNER JOIN albums, genres
  ON tracks.AlbumID = albums.AlbumID and tracks.GenreID = genres.GenreID
  WHERE albums.AlbumID IN (
    SELECT AlbumID
    FROM tracks
    GROUP BY AlbumID
    HAVING COUNT(DISTINCT GenreID) > 1
  ))
GROUP BY AlbumsName, GenresName
ORDER BY AlbumsName

 * sqlite:///chinook.db
Done.


AlbumsName,GenresName,COUNT(AlbumsName)
"Battlestar Galactica, Season 3",Sci Fi & Fantasy,2
"Battlestar Galactica, Season 3",Science Fiction,12
"Battlestar Galactica, Season 3",TV Shows,5
Greatest Hits,Metal,14
Greatest Hits,Reggae,13
Greatest Hits,Rock,30
"Heroes, Season 1",Drama,20
"Heroes, Season 1",TV Shows,3
"LOST, Season 4",Drama,14
"LOST, Season 4",TV Shows,3


## Задание 5
Для каждого артиста (таблицы Artists), у которых есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение суммы, заработанной артистом по этому жанру к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, максимальное для артиста, то выбирается жанр раньше по алфавиту. Формат вывода: артист, жанр, соотношение.

In [9]:
%%sql
SELECT ArtistName as "Артист", GenreName as "Жанр",
MAX(UnitPriceSum) as "Сумма", MAX(UnitPriceSum) / SUM(UnitPriceSum) as "Соотношение"
FROM
(
  SELECT artists.Name as ArtistName, genres.Name as GenreName,
  SUM(tracks.UnitPrice) as UnitPriceSum
  FROM artists
  INNER JOIN albums, tracks, genres
  ON artists.ArtistID = albums.ArtistID and
    albums.AlbumID = tracks.AlbumID and
    genres.GenreId = tracks.GenreID

  WHERE artists.ArtistID IN (
    SELECT artists.ArtistID
    FROM artists
    INNER JOIN albums, tracks, genres
    ON artists.ArtistID = albums.ArtistID and
      albums.AlbumID = tracks.AlbumID and
      genres.GenreId = tracks.GenreID
    GROUP BY artists.ArtistID
    HAVING COUNT(DISTINCT genres.GenreId) > 1
  )
  GROUP BY artists.ArtistID, genres.GenreID
)
GROUP BY ArtistName

 * sqlite:///chinook.db
Done.


Артист,Жанр,Сумма,Соотношение
Amy Winehouse,R&B/Soul,11.88,0.5217391304347826
Antônio Carlos Jobim,Latin,16.830000000000002,0.5483870967741935
Audioslave,Rock,13.860000000000001,0.35000000000000003
Battlestar Galactica,Science Fiction,25.869999999999994,0.65
Eric Clapton,Blues,31.67999999999998,0.6666666666666665
Faith No More,Alternative & Punk,36.62999999999999,0.7115384615384615
Foo Fighters,Rock,32.66999999999998,0.7499999999999999
Gilberto Gil,Latin,14.850000000000001,0.46875000000000006
Guns N' Roses,Rock,27.719999999999985,0.6666666666666665
Heroes,Drama,39.8,0.8695652173913043
